In [66]:
import os
import sys
import pickle
import numpy as np
from sklearn.metrics import mean_squared_error

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from bayesmf.scripts.utils.model import run_kfold_xval
from bayesmf.scripts.utils.dataset import make_insilico_dataset, make_downsampled_dataset
from bayesmf.models.lda import LDA, StochasticLDA
from bayesmf.models.cmf import CMF, StochasticCMF

%reload_ext autoreload
%autoreload 2

In [67]:
DIR_PATH = '/home/sxchao/bayesmf/output'
handle = 'scmf_050720'

In [68]:
X, _ = make_insilico_dataset()

In [92]:
np.random.seed(22690)

idxs = np.arange(X.shape[1])
np.random.shuffle(idxs)

X_train = X[:, idxs[:900]]
X_test = X[:, idxs[900:]]

In [93]:
factorizer = LDA(K=15, random_state=22690, init='nmf', verbose=True)
factorizer.fit(X_train.T) # V x D -> D x V

/home/sxchao/bayesmf/models/lda.py:139: RuntimeWarning: invalid value encountered in double_scalars
  chg = (elbo_new - elbo_old) / abs(elbo_old)


Iter: 0, Bound: 19969.78, Change: nan
Iter: 1, Bound: 20143.48, Change: 0.00870
Iter: 2, Bound: 20247.61, Change: 0.00517
Iter: 3, Bound: 20324.54, Change: 0.00380
Iter: 4, Bound: 20393.85, Change: 0.00341
Iter: 5, Bound: 20426.11, Change: 0.00158
Iter: 6, Bound: 20465.09, Change: 0.00191
Iter: 7, Bound: 20497.98, Change: 0.00161
Iter: 8, Bound: 20506.63, Change: 0.00042


LDA(K=15, init='nmf', max_iters=100, random_state=22690, smoothness=100,
    tolerance=0.0005, verbose=True)

In [94]:
W = factorizer.transform(X_test.T, attr='Et') # D x K
H = factorizer.Eb # K x V
X_pred = np.matmul(W * np.sum(X_test, axis=0)[:,np.newaxis],H)
mean_squared_error(X_test.T, X_pred, squared=False)

/home/sxchao/bayesmf/models/lda.py:139: RuntimeWarning: invalid value encountered in double_scalars
  chg = (elbo_new - elbo_old) / abs(elbo_old)


1.317133053646476

In [58]:
kfold = 5
splits = np.split(idxs, kfold)

In [ ]:
K = 10
errs10 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]
    
    factorizer = StochasticCMF(K=K, m=K-5, minibatch_size=200, step_size=1e-05, tolerance=0.001, max_iters=10, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T
        
    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    errs10.append(err)

In [ ]:
K = 15
errs15 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]

    factorizer = StochasticCMF(K=K, m=K-5, step_size=1e-05, tolerance=0.001, max_iters=10, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T

    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    errs15.append(err)

In [ ]:
K = 20
errs20 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]

    factorizer = StochasticCMF(K=K, m=K-5, step_size=1e-05, tolerance=0.001, max_iters=10, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T

    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    errs20.append(err)

In [41]:
X, _ = make_downsampled_dataset()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [42]:
np.random.seed(22690)

idxs = np.arange(X.shape[1])
np.random.shuffle(idxs)

#X_train = X[:, idxs[:800]]
#X_test = X[:, idxs[800:]]

kfold = 5
splits = np.split(idxs, kfold)

In [ ]:
K = 10
derrs10 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]
    
    factorizer = StochasticCMF(K=K, m=K-5, step_size=1e-05, tolerance=0.001, max_iters=10, init=None, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T
        
    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    derrs10.append(err)

In [ ]:
K = 15
derrs15 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]

    factorizer = StochasticCMF(K=K, m=K-5, step_size=1e-05, tolerance=0.001, max_iters=10, init=None, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T

    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    derrs15.append(err)

In [ ]:
K = 20
derrs20 = []

for k in range(kfold):
    idxs_train = [i for j in np.setdiff1d(np.arange(kfold), k) for i in splits[j]]
    idxs_test = splits[k]
    X_train = X[:, idxs_train]
    X_test = X[:, idxs_test]

    factorizer = StochasticCMF(K=K, m=K-5, step_size=1e-05, tolerance=0.001, max_iters=10, init=None, kwargs={'c0':0.05 * X.shape[0]})
    factorizer.fit(X_train)
    l = factorizer.transform(X_test, attr='l') 
    W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T 
    H = factorizer.Eb.T

    err = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
    print(err)
    derrs20.append(err)

In [48]:
# init=None, synthetic dataset
for errs in [errs10, errs15, errs20]:
    print(np.mean(errs))

1.5734142857619882
1.5727257711655738
1.5724411292628946


In [49]:
# init=None, downsampled dataset
for errs in [derrs10, derrs15, derrs20]:
    print(np.mean(errs))

4.300839159962477
4.2855199746437
4.286197600236283
